# Import Dependencies

In [1]:
import argparse
import io
import logging
import pickle

import jsonpickle
import numpy as np
import requests
from PIL import Image, ImageDraw, ImageFont

import io
import logging


import numpy as np
import torch
from tqdm import tqdm

from model.model import ResMLP
from utils import enable_dropout, forward_mc, read_json

from flask import Flask, request
import logging
from insightface.app.face_analysis import FaceAnalysis as FaceDetectionRecognition
import numpy as np
import argparse
import io
from PIL import Image
from utils import resize_square_image, get_original_bbox, get_original_lm


/home/jacky/Downloads/temporary/age-gender-main/age-gender/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Start Face Detection Model Flask Server

In [2]:
app = Flask(__name__)

# gender age estimaion are bad. We don't use them here.
fdr = FaceDetectionRecognition(det_name='retinaface_r50_v1',
                               rec_name='arcface_r100_v1',
                               ga_name=None)


@app.route("/", methods=["POST"])
def face_detection_recognition():
    """Receive everything in json!!!
    """
    app.logger.debug(f"Receiving data ...")
    data = request.json
    data = jsonpickle.decode(data)

    app.logger.debug(f"decompressing image ...")
    image = data['image']
    image = io.BytesIO(image)

    app.logger.debug(f"Reading a PIL image ...")
    image = Image.open(image)
    image_size_original = image.size

    app.logger.debug(f"Resizing a PIL image to 640 by 640 ...")
    image = resize_square_image(image, 640, background_color=(0, 0, 0))
    image_size_new = image.size

    app.logger.debug(f"Conveting a PIL image to a numpy array ...")
    image = np.array(image)

    if len(image.shape) != 3:
        app.logger.error(f"image shape: {image.shape} is not RGB!")
        del data, image
        response = {'face_detection_recognition': None}
        response_pickled = jsonpickle.encode(response)
        return response_pickled

    app.logger.info(f"extraing features ...")
    list_of_features = fdr.get(image)
    app.logger.info(f"features extracted!")

    app.logger.info(f"In total of {len(list_of_features)} faces detected!")

    results_frame = []
    for features in list_of_features:
        bbox = get_original_bbox(features.bbox, image_size_original, image_size_new)
        landmark = get_original_lm(features.landmark, image_size_original, image_size_new)
        feature_dict = {'bbox': bbox,
                        'det_score': features.det_score,
                        'landmark': landmark,
                        'normed_embedding': features.normed_embedding
                        }
        results_frame.append(feature_dict)

    response = {'face_detection_recognition': results_frame}
    app.logger.info("json-pickle is done.")

    response_pickled = jsonpickle.encode(response)

    return response_pickled


if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--gpu-id', type=int, default=-1, help='-1 means CPU')
    # args = parser.parse_args()
    # args = vars(args)
    # fdr.prepare(ctx_id=args['gpu_id'])
    app.run(host='0.0.0.0', port=10002)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jacky/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jacky/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jacky/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jacky/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /home/jacky/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
 * Serv

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:10002
 * Running on http://192.168.1.6:10002
Press CTRL+C to quit
[2022-12-02 19:29:44,593] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "/home/jacky/Downloads/temporary/age-gender-main/age-gender/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/jacky/Downloads/temporary/age-gender-main/age-gender/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/jacky/Downloads/temporary/age-gender-main/age-gender/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/jacky/Downloads/temporary/age-gender-main/age-gender/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  Fil

: 